# Diffusion Training and Experimentation

This notebook demonstrates training Diffusion model on the MNIST dataset using a modular training pipeline implemented in PyTorch Lightning.


In [1]:
from src.models.mnist_vae import MnistVAE
!git clone https://github.com/Reennon/gen-ai-cv-2-3.git
%cd gen-ai-cv-2-3

fatal: destination path 'gen-ai-cv-2-3' already exists and is not an empty directory.
/content/gen-ai-cv-2-3


In [2]:
!pip install -r requirements.txt

In [3]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5 (delta 2), reused 5 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 1.08 KiB | 1.08 MiB/s, done.
From https://github.com/Reennon/gen-ai-cv-2-3
   5a62562..b17cc25  main       -> origin/main
Updating 5a62562..b17cc25
Fast-forward
 src/training/diffusion_module.py | 35 +++++++++++++++++++++++++++++++++++
 1 file changed, 35 insertions(+)


In [4]:
import os
import dotenv
import wandb
import torch

from omegaconf import OmegaConf
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from src.training.latent_diffusion_module import (LatentDiffusionModel)
from src.training.trainer import train_model

from google.colab import userdata

In [6]:
os.environ["WANDB_KEY"] = userdata.get("wandb_key")
!echo $WANDB_KEY >> .env

In [7]:
dotenv.load_dotenv()

True

In [8]:
parameters = OmegaConf.load("./params/latent_diffusion.yml")
wandb.login(key=os.environ["WANDB_KEY"])

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rkovalch (rkovalchuk). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
wandb_project_name = "latent-diffusion-mnist"
device = "gpu"

In [10]:
# Prepare the MNIST dataset.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_train = MNIST(root='data', train=True, download=True, transform=transform)
mnist_val = MNIST(root='data', train=False, download=True, transform=transform)
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=64)


In [17]:
from src.training.latent_diffusion_module import sample_ddim

vae = MnistVAE()

import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=10, accelerator="auto")

print("Fitting MNIST VAE ---")
trainer.fit(vae, train_loader)

ddpm = LatentDiffusionModel(
    pretrained_vae=vae
)


print("Fitting DDPM ---")

trainer.fit(ddpm, train_loader)

generated_images = sample_ddim(ddpm, vae, sample_count=32)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params | Mode 
---------------------------------------------------------------
0 | model | SimpleUnconditionalDiffusion | 47.0 K | train
---------------------------------------------------------------
47.0 K    Trainable params
0         Non-trainable params
47.0 K    Total params
0.188     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [ ]:
import matplotlib.pyplot as plt

def display_images(img_tensor):
    # Convert tensor to NumPy array for plotting
    img_array = img_tensor.detach().cpu().numpy()

    # Create a grid of 4x4 subplots
    fig, axs = plt.subplots(nrows=8, ncols=4, figsize=(12, 12))

    # Loop through each subplot and display an image in grayscale
    for idx, ax in enumerate(axs.flat):
        ax.imshow(img_array[idx, 0], cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()
    plt.show()

display_images(generated_images)


In [14]:
wandb.finish()

epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇█
learning_rate_optimizer_0,▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▅▃▂▂▅▃▃▃▃▂▂▂▃▁▁▃▂▃▂▂▁▁▂▂▁▁▁▁▂▂▂▁▁▁▂▂▂▁
trainer/global_step,▁▁▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇████
val_loss,█▃▄▃▃▁▂▂▁▁
epoch,9
learning_rate_optimizer_0,0.001
train_loss,0.03237
trainer/global_step,9379
val_loss,0.03204


In [15]:
# from google.colab import runtime
# runtime.unassign()